In [6]:

import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("app.log"),          # File output
        logging.StreamHandler()                  # Console output
    ],
    force=True  # This overrides any prior logging config
)

import clickhouse_connect
import requests
import pandas as pd

import sys
import os

from pydantic import BaseModel, Field

from typing import List
import os
import uuid

from dotenv import load_dotenv
load_dotenv()


CLICKHOUSE_USER=os.getenv("CLICKHOUSE_USER")
CLICKHOUSE_PASSWORD=os.getenv("CLICKHOUSE_PASSWORD")
CH_IP=os.getenv("CH_IP")
CH_PORT=os.getenv("CH_PORT")  
CLICKHOUSE_DB=os.getenv("CLICKHOUSE_DB") 

# Initialize ClickHouse client
client_ch = clickhouse_connect.get_client(
    host=CH_IP,
    port=CH_PORT,     
    username=CLICKHOUSE_USER,
    password=CLICKHOUSE_PASSWORD,
    database=CLICKHOUSE_DB
)


# import tools
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# find tools in parent dir
if os.path.isdir(os.path.join(parent_dir, 'tools')):
    # Add parent directory to sys.path if found
    sys.path.append(parent_dir)
    
else:
    # for run in spark
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../airflow/airflow_data"))
    
    # Add parent directory to sys.path
    sys.path.append(parent_dir)


from tools import pd_tools
from tools.paths import Paths
from tools.db_tools import DbTools




In [7]:
air_quality = pd.read_csv('data/Air_Quality.csv')
air_quality.head()

,Unique ID,Indicator ID,name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Unnamed: 10,Data Value
0,419355,365,Fine particles (PM 2.5),Estimated annual rate,per square mile,Citywide,208.0,St. George and Stapleton (CD1),Summer 2010,12/31/2022,NaN,0
1,542128,365,Fine particles (PM 2.5),Estimated annual rate (age 30+),"per 100,000",Borough,404406.0,Northern SI,Summer 2010,12/31/2022,NaN,47
2,419346,640,Nitrogen dioxide (NO2),Number per km2,"per 100,000",CD,307.0,Fresh Meadows,Summer 2011,31.12.2022,NaN,0
3,419347,365,Boiler Emissions- Total PM2.5 Emissions,Estimated annual rate (age 30+),"per 100,000 adults",UHF42,305.0,Southeast Queens,Annual Average 2009,31.12.2022,NaN,0
4,419348,367,Ozone (O3),Estimated annual rate (age 30+),ppb,UHF42,105.0,Greenpoint and Williamsburg (CD1),Annual Average 2011,31.12.2022,NaN,0


In [9]:
air_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134788 entries, 0 to 134787
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unique ID       134788 non-null  int64  
 1   Indicator ID    134788 non-null  int64  
 2   name            134788 non-null  object 
 3   Measure         134788 non-null  object 
 4   Measure Info    134788 non-null  object 
 5   Geo Type Name   134788 non-null  object 
 6   Geo Join ID     134779 non-null  float64
 7   Geo Place Name  134779 non-null  object 
 8   Time Period     134788 non-null  object 
 9   Start_Date      134788 non-null  object 
 10  Unnamed: 10     0 non-null       float64
 11  Data Value      134788 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 12.3+ MB


In [10]:
air_quality = air_quality.drop(columns='Unnamed: 10')

In [11]:
pd_tools.df_info(air_quality)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134788 entries, 0 to 134787
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unique ID       134788 non-null  int64  
 1   Indicator ID    134788 non-null  int64  
 2   name            134788 non-null  object 
 3   Measure         134788 non-null  object 
 4   Measure Info    134788 non-null  object 
 5   Geo Type Name   134788 non-null  object 
 6   Geo Join ID     134779 non-null  float64
 7   Geo Place Name  134779 non-null  object 
 8   Time Period     134788 non-null  object 
 9   Start_Date      134788 non-null  object 
 10  Data Value      134788 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 11.3+ MB
None


'First 5 rows in df'

,Unique ID,Indicator ID,name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value
0,419355,365,Fine particles (PM 2.5),Estimated annual rate,per square mile,Citywide,208.0,St. George and Stapleton (CD1),Summer 2010,12/31/2022,0
1,542128,365,Fine particles (PM 2.5),Estimated annual rate (age 30+),"per 100,000",Borough,404406.0,Northern SI,Summer 2010,12/31/2022,47
2,419346,640,Nitrogen dioxide (NO2),Number per km2,"per 100,000",CD,307.0,Fresh Meadows,Summer 2011,31.12.2022,0
3,419347,365,Boiler Emissions- Total PM2.5 Emissions,Estimated annual rate (age 30+),"per 100,000 adults",UHF42,305.0,Southeast Queens,Annual Average 2009,31.12.2022,0
4,419348,367,Ozone (O3),Estimated annual rate (age 30+),ppb,UHF42,105.0,Greenpoint and Williamsburg (CD1),Annual Average 2011,31.12.2022,0


Column Unique ID has only one type: <class 'int'>
Column Indicator ID has only one type: <class 'int'>
Column name has only one type: <class 'str'>
Column Measure has only one type: <class 'str'>
Column Measure Info has only one type: <class 'str'>
Column Geo Type Name has only one type: <class 'str'>
Column Geo Join ID has only one type: <class 'float'>
Column Geo Place Name has only one type: <class 'str'>
Column Time Period has only one type: <class 'str'>
Column Start_Date has only one type: <class 'str'>
Column Data Value has only one type: <class 'str'>


,duplicates_full
duplicates_full,0


...

""
nones


""
NA placeholder


""
null placeholder


""
N/A placeholder


""
